# Mongo Agent

In [2]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, CodeAgent
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Optional

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL_ID = "gpt-4o-mini"
MONGODB_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
debug = os.getenv('DEBUG', 'False').lower() == 'true'

In [5]:
mongoclient = MongoClient(MONGODB_URI, appname="mongo-agent")
db = mongoclient.warehouse

In [6]:
@tool
def check_stock(product_id: str) -> Dict:
    """Query product stock level.

    Args:
        product_id: Product identifier

    Returns:
        Dict containing product details and quantity
    """
    return db.products.find_one({"_id": product_id})


@tool
def update_stock(product_id: str, quantity: int) -> bool:
    """Update product stock quantity.

    Args:
        product_id: Product identifier
        quantity: Amount to decrease from stock

    Returns:
        bool: Success status
    """
    result = db.products.update_one({"_id": product_id}, {"$inc": {"quantity": -quantity}})
    return result.modified_count > 0

In [7]:
@tool
def create_order(products: any, address: str) -> str:
    """Create new order for all provided products.

    Args:
        products: List of products with quantities
        address: Delivery address

    Returns:
        str: Order ID message
    """
    order = {"products": products, "status": "pending", "delivery_address": address, "created_at": datetime.now()}
    result = db.orders.insert_one(order)
    return f"Successfully ordered : {str(result.inserted_id)}"

In [8]:
from bson.objectid import ObjectId


@tool
def update_delivery_status(order_id: str, status: str) -> bool:
    """Update order delivery status to in_transit once a pending order is provided

    Args:
        order_id: Order identifier
        status: New delivery status is being set to in_transit or delivered

    Returns:
        bool: Success status
    """
    if status not in ["pending", "in_transit", "delivered", "cancelled"]:
        raise ValueError("Invalid delivery status")

    result = db.orders.update_one({"_id": ObjectId(order_id), "status": "pending"}, {"$set": {"status": status}})
    return result.modified_count > 0